## **Run the cell below to train the grading BERT-based model**

In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

def train_bert_for_grading(
    json_path, 
    output_dir="./bert_grade_model", 
    num_epochs=3, 
    batch_size=16, 
    learning_rate=2e-5, 
    weight_decay=0.0,
    hf_repo_name="CommitGraderModel"
):
    """
    Trains a BERT model for sequence classification on commit messages and grades.

    Args:
        json_path (str): Path to the JSON dataset file with `commit_message` and `grade` fields.
        output_dir (str): Directory to save the trained model and tokenizer locally.
        num_epochs (int): Number of epochs for training.
        batch_size (int): Batch size for training.
        learning_rate (float): Learning rate for the optimizer.
        weight_decay (float): Weight decay for the optimizer.
        hf_repo_name (str): Hugging Face repo name to push the model to.
    """

    # Load the dataset
    dataset = load_dataset("json", data_files=json_path)["train"]

    # Tokenizer and model initialization
    tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
    model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=4)

    # Preprocessing function
    def preprocess_function(examples):
        inputs = examples["commit_message"]
        targets = examples["grade"]
        tokenized_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=128)
        tokenized_inputs["labels"] = targets
        return tokenized_inputs

    # Apply preprocessing
    tokenized_dataset = dataset.map(preprocess_function, batched=True)

    # Split the dataset into train and test sets
    def train_test_split_dataset(dataset, test_size=0.2):
        df = dataset.to_pandas()
        train_df, test_df = train_test_split(df, test_size=test_size, random_state=42)
        train_dataset = Dataset.from_pandas(train_df)
        eval_dataset = Dataset.from_pandas(test_df)
        return train_dataset, eval_dataset

    train_dataset, eval_dataset = train_test_split_dataset(tokenized_dataset)

    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=weight_decay,
        logging_dir="./logs",
        logging_steps=10,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        push_to_hub=True,
        hub_model_id=hf_repo_name,
        hub_strategy="every_save"
    )

    # Metric computation
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = torch.argmax(torch.tensor(logits), axis=1)
        accuracy = (predictions == torch.tensor(labels)).float().mean().item()
        return {"accuracy": accuracy}

    # Trainer initialization
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Save locally
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Push final versions to Hugging Face
    model.push_to_hub(hf_repo_name, private=False)
    tokenizer.push_to_hub(hf_repo_name)

    print(f"✅ Training complete! Model and tokenizer saved locally to '{output_dir}' and pushed to Hugging Face as '{hf_repo_name}'.")

# Call the function
train_bert_for_grading(
    json_path="training_data.json",
    output_dir="./bert_grade_model",
    num_epochs=3,
    batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.0,
    hf_repo_name="CommitGraderModel"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/s9/qzfnyl8j1hqb_tsw2bpl5bbc0000gn/T/ipykernel_22487/1989764975.py:84: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.542400,0.593584,0.749593
2,0.531400,0.538104,0.769106
3,0.372300,0.539527,0.778862


/opt/anaconda3/envs/swent/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/swent/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
No files have been modified since last commit. Skipping to prevent empty commit.


✅ Training complete! Model and tokenizer saved locally to './bert_grade_model' and pushed to Hugging Face as 'CommitGraderModel'.
